# Down-scaled multi-area model

#### Notebook structure <a class="anchor" id="toc"></a>
* [S0. Configuration](#section_0)
* [S1. Paramters specification](#section_1)
    * [1.1. Parameters to tune](#section_1_1)
    * [1.2. Default parameters](#section_1_2)
* [S2. Multi-area model instantiation and simulation](#section_2)
    * [2.1. Insantiate a multi-area model](#section_2_1)
    * [2.2. Predict firing rates from theory](#section_2_2)
    * [2.3. Extract connectivity](#section_2_3)
    * [2.4. Run the simulation](#section_2_4)
* [S3. Data processing and simulation results analysis](#section_3)
* [S4. Simulation results visualization](#section_4) 

<br>

## S0. Configuration <a class="anchor" id="section_0"></a>

In [ ]:
# Import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nest
from IPython.display import display, HTML

# Import the MultiAreaModel class
from multiarea_model import MultiAreaModel
from config import base_path

In [ ]:
# Create config file
with open('config.py', 'w') as fp:
    fp.write(
'''import os
base_path = os.path.abspath(".")
data_path = os.path.abspath("simulations")
jobscript_template = "python {base_path}/run_simulation.py {label}"
submit_cmd = "bash -c"
''')

In [ ]:
!pip install nested_dict dicthash

In [ ]:
# Jupyter notebook display format setting
style = """
<style>
table {float:left}
</style>
"""
display(HTML(style))

Go back to [Notebook structure](#toc)

<br>

## S1. Paramters specification <a class="anchor" id="section_1"></a>

### 1.1. Parameters to tune <a class="anchor" id="section_1_1"></a>

|Parameter                         |Variable                    |Value               |Value description  |
|:--------------------------------:|:--------------------------:|:------------------:|:------------------|
|Scaling factor                    |scale_down_to               |0.005               |$^1$               |
|Ground state to metastable state  |cc_weights_factor           |                    |$^2$               |
|Replace non-simulated areas       |replace_non_simulated_areas |'het_poisson_stat'  |$^3$               |
|Simulation areas                  |sim_areas                   |                    |$^4$               |

1. Scaling factor (`scale_down_to`) <br>
Scaling factor (scale_down_to) is the parameter which defines the the ratio of the full scale multi-area model being down-scaled to a model with fewer neurons and indegrees so as to be simulated on machines with lower computational ability and the simulation results can be obtained within relative shorter period of time.
Neurons and indegrees are both scaled down to 0.5%, where the model can usually be simulated on a local machine.<br> 
**Warning**: This will not yield reasonable dynamical results from the network and is only meant to demonstrate the simulation workflow <br> 
2. Ground state to metastable state (`cc_weights_factor`) <br>
Ground state to metastable state (cc_weights_factor) decides the switch of the model from ground state to metastable state. <br>
3. Replace non-simulated areas (`replace_non_simulated_areas`) <br>
Replace non-simulated areas (replace_non_simulated_areas) defines how non-simulated areas will be replaced. <br>
4. Simulation areas (`simulation_areas`) <br>
Simulation areas (simulation_areas) specify the cortical areas included in the simulation process.

In [ ]:
# Model parameters
# Model paramters are most important among all the paramters, it directly affect the model itself and thus have a great impact on the simulation results. Model paramters define the connection, input, neuron, and network charateristics of the model, and therefore fall into four categories: **Connection paramters**, **Input paramters**, **Neuron paramters**, and **Network paramters**.
scale_down_to = 0.005 # Change it to 1. for running the fullscale network
# Scaling factor for cortico-cortical connections (chi) 
# This scaling factor controls the cortico-cortical synaptic strength. If it is 1.0 then the inter-area synaptic strength is the same as the intra-areal. This factor changes the network activity from ground to metastable.
cc_weights_factor = 1.
replace_non_simulated_areas = 'het_poisson_stat'
sim_areas = 

### 1.2. Default parameters <a class="anchor" id="section_1_2"></a>
Default parameters ...<br>
We try out best not to confuse users with too many parameters. However, if you want to change the default parameters, you can do so by passing a dictionary to the `default_params` argument of the `MultiAreaModel` class.

In [ ]:
# Connection parameters (conn_params)
conn_params = {
    # # It defines how non-simulated areas will be replaced
    # Whether to replace non-simulated areas by Poisson sources with the same global rate rate_ext ('hom_poisson_stat') or by specific rates ('het_poisson_stat') or by time-varying specific current ('het_current_nonstat'). In the two latter cases, the data to replace the cortico-cortical input is loaded from `replace_cc_input_source`
    'replace_non_simulated_areas': 'het_poisson_stat', 

    'g': -11.,
    'K_stable': 'K_stable.npy',
    # Increase the external input to 2/3E and 5E in area TH
    'fac_nu_ext_TH': 1.2,
    # Increase the external Poisson indegree onto 5E
    'fac_nu_ext_5E': 1.125,
    # Increase the external Poisson indegree onto 6E
    'fac_nu_ext_6E': 1.41666667,
    # Adjust the average indegree in V1 based on monkey data
    'av_indegree_V1': 3950.
}

# Input parameters (input_params)
input_params = {'rate_ext': 10.}

# Neuron parameters (neuron_params)
neuron_params = {'V0_mean': -150.,
                 'V0_sd': 50.}

# Network parameters (network_params)
network_params = {'N_scaling': scale_down_to,
                  'K_scaling': scale_down_to,
                  'fullscale_rates': 'tests/fullscale_rates.json',
                  'input_params': input_params,
                  'connection_params': conn_params,
                  'neuron_params': neuron_params}

# Simulation parameters (sim_params)
sim_params = {'t_sim': 2000.,
              'num_processes': 1,
              'local_num_threads': 1,
              'recording_dict': {'record_vm': False},
              'rng_seed': 1} # global random seed

# Theory paramters (theory_params)
theory_params = {'dt': 0.1}

Go back to [Notebook structure](#toc)

<br>

## S2. Multi-area model instantiation and simulation <a class="anchor" id="section_2"></a>

### 2.1. Insantiate a multi-area model <a class="anchor" id="section_2_1"></a>

In [ ]:
M = MultiAreaModel(network_params, simulation=True,
                   sim_spec=sim_params,
                   theory=True,
                   theory_spec=theory_params)

### 2.2. Predict firing rates from theory <a class="anchor" id="section_2_2"></a>

In [ ]:
p, r = M.theory.integrate_siegert()
print("Mean-field theory predicts an average "
      "rate of {0:.3f} spikes/s across all populations.".format(np.mean(r[:, -1])))

### 2.3. Extract connectivity <a class="anchor" id="section_2_3"></a>

The connectivity and neuron numbers are stored in the attributes of the model class. Neuron numbers are stored in `M.N` as a dictionary (and in `M.N_vec` as an array), indegrees in `M.K` as a dictionary (and in `M.K_matrix` as an array). Number of synapses can also be access via `M.synapses` (and in `M.syn_matrix` as an array). <br>

**Warning**: memory explosion

#### 2.3.1 Node indegrees

In [ ]:
# Dictionary of nodes indegrees organized as:
# {<source_area>: {<source_pop>: {<target_area>: {<target_pop>: indegree_values}}}}
# M.K

#### 2.3.2 Synapses

In [ ]:
# Dictionary of synapses that target neurons receive, it is organized as:
# {<source_area>: {<source_pop>: {<target_area>: {<target_pop>: number_of_synapses}}}}
# M.synapses

Go back to [Notebook structure](#toc)

<br>

### 2.4. Run the simulation <a class="anchor" id="section_2_4"></a>

In [ ]:
# run the simulation, depending on the model parameter and downscale ratio, the running time varies largely.
M.simulation.simulate()

Go back to [Notebook structure](#toc)

<br>

## S3. Data processing and simulation results analysis <a class="anchor" id="section_3"></a>

The following instructions will work when the `simulate` parameter is set to `True` during the creation of the MultiAreaModel object, and the `M.simulation.simulate()` method is executed.

In [ ]:
# Uncomment the lines in this code cell below to test if the number of synapses created by NEST matches the expected values

# """
# Test if the correct number of synapses has been created.
# """
# print("Testing synapse numbers")
# for target_area_name in M.area_list:
#     target_area = M.simulation.areas[M.simulation.areas.index(target_area_name)]
#     for source_area_name in M.area_list:
#         source_area = M.simulation.areas[M.simulation.areas.index(source_area_name)]
#         for target_pop in M.structure[target_area.name]:
#             target_nodes = target_area.gids[target_pop]
#             for source_pop in M.structure[source_area.name]:
#                 source_nodes = source_area.gids[source_pop]
#                 created_syn = nest.GetConnections(source=source_nodes,
#                                                   target=target_nodes)
#                 syn = M.synapses[target_area.name][target_pop][source_area.name][source_pop]
#                 assert(len(created_syn) == int(syn))

To obtain the connections information, you can extract the lists of connected sources and targets. Moreover, you can access additional synaptic details, such as synaptic weights and delays.

In [ ]:
# conns = nest.GetConnections()
# conns_sparse_matrix = conns.get(['source', 'target', 'weight'])

# srcs = conns_sparse_matrix['source']
# tgts = conns_sparse_matrix['target']
# weights = conns_sparse_matrix['weight']

You can determine the area and subpopulation to which the neuron ID ranges belong by referring to the file `network_gids.txt`, which is automatically generated during network creation.

In [ ]:
# # Open the file using a with statement
# with open(os.path.join(M.simulation.data_dir,"recordings/network_gids.txt"), "r") as file:
#     # Read the contents of the file
#     gids = file.read()

# # Print the contents
# print(gids)

### 1. Load spike data

In [ ]:
data = np.loadtxt(M.simulation.data_dir + '/recordings/' + M.simulation.label + "-spikes-1-0.dat", skiprows=3)

### 2. Compute instantaneous rate per neuron across all populations

In [ ]:
tsteps, spikecount = np.unique(data[:,1], return_counts=True)
rate = spikecount / M.simulation.params['dt'] * 1e3 / np.sum(M.N_vec)

Go back to [Notebook structure](#toc)

<br>

## S4. Simulation results visualization <a class="anchor" id="section_4"></a>

### 4.1. Instantaneous and mean rate

In [ ]:
fig, ax = plt.subplots()
ax.plot(tsteps, rate)
ax.plot(tsteps, np.average(rate)*np.ones(len(tsteps)), label='mean')
ax.set_title('instantaneous rate across all populations')
ax.set_xlabel('time (ms)')
ax.set_ylabel('rate (spikes / s)')
ax.set_xlim(0, sim_params['t_sim'])
ax.set_ylim(0, 50)
ax.legend()

### 4.2 Resting state for single area
Raster plot of spiking activity of 3% of the neurons in area V1 (A), V2 (B), and FEF (C). Blue: excitatory neurons, red: inhibitory neurons. (D-F) Spiking statistics across all 32 areas for the respective populations shown as area-averaged box plots. Crosses: medians, boxes: interquartile range (IQR), whiskers extend to the most extremeobservat ions within 1.5×IQR beyond the IQR.

In [ ]:
"""
Create raster display of a single area with populations stacked
onto each other. Excitatory neurons in blue, inhibitory
neurons in red.

Parameters
----------
area : string {area}
    Area to be plotted.
frac_neurons : float, [0,1]
    Fraction of cells to be considered.
t_min : float, optional
    Minimal time in ms of spikes to be shown. Defaults to 0 ms.
t_max : float, optional
    Minimal time in ms of spikes to be shown. Defaults to simulation time.
output : {'pdf', 'png', 'eps'}, optional
    If given, the function stores the plot to a file of the given format.

"""
area = 'V1'
frac_neurons = 0.03
M.analysis.single_dot_display(area,  frac_neurons, t_min=500., t_max='T')

area = 'V2'
frac_neurons = 0.03
M.analysis.single_dot_display(area,  frac_neurons, t_min=500., t_max='T')

area = 'FEF'
frac_neurons = 0.03
M.analysis.single_dot_display(area,  frac_neurons, t_min=500., t_max='T')

### 4.3 Firing rates for the whole population
Population-averaged firing rates

In [ ]:
"""
Calculate time-averaged population rates and store them in member pop_rates.
If the rates had previously been stored with the same
parameters, they are loaded from file.

Parameters
----------
t_min : float, optional
    Minimal time in ms of the simulation to take into account
    for the calculation. Defaults to 500 ms.
t_max : float, optional
    Maximal time in ms of the simulation to take into account
    for the calculation. Defaults to the simulation time.
compute_stat : bool, optional
    If set to true, the mean and variance of the population rate
    is calculated. Defaults to False.
    Caution: Setting to True slows down the computation.
areas : list, optional
    Which areas to include in the calculcation.
    Defaults to all loaded areas.
pops : list or {'complete'}, optional
    Which populations to include in the calculation.
    If set to 'complete', all populations the respective areas
    are included. Defaults to 'complete'.
"""
M.analysis.create_pop_rates(t_min=1000.)
# M.analysis.save()

### 4.4 Average pairwise correlation coefficients of spiking activity

### 4.5 Irregularity measured by revised local variation LvR averaged across neurons

### 4.6 Time series of population- and area-averaged firing rates.
Area-averaged firing rates, shown as raw binned spike histograms with 1ms bin width (gray) and convolved histograms, with aGaussian kernel (black) of optimal width

In [ ]:
"""
Calculate time series of population- and area-averaged firing rates.
Uses ah.pop_rate_time_series.
If the rates have previously been stored with the
same parameters, they are loaded from file.


Parameters
----------
t_min : float, optional
    Minimal time in ms of the simulation to take into account
    for the calculation. Defaults to 500 ms.
t_max : float, optional
    Maximal time in ms of the simulation to take into account
    for the calculation. Defaults to the simulation time.
areas : list, optional
    Which areas to include in the calculcation.
    Defaults to all loaded areas.
pops : list or {'complete'}, optional
    Which populations to include in the calculation.
    If set to 'complete', all populations the respective areas
    are included. Defaults to 'complete'.
kernel : {'gauss_time_window', 'alpha_time_window', 'rect_time_window'}, optional
    Specifies the kernel to be convolved with the spike histogram.
    Defaults to 'binned', which corresponds to no convolution.
resolution: float, optional
    Width of the convolution kernel. Specifically it correponds to:
    - 'binned' : bin width of the histogram
    - 'gauss_time_window' : sigma
    - 'alpha_time_window' : time constant of the alpha function
    - 'rect_time_window' : width of the moving rectangular function
"""
M.analysisi.create_rate_time_series(t_max=1000.)
# M.analysis.save()

Go back to [Notebook structure](#toc)